In [182]:
import numpy
import re

file = open('data/17.txt', 'r')
data = file.read()

file.close()

In [183]:
cave = numpy.zeros(shape=(2022*4, 7), dtype=numpy.int8)
gas = list(map(lambda c: ord(c) - 61, data))
current_y = cave.shape[0]

shapes = [
    numpy.array([
        [1, 1, 1, 1]
    ]),
    numpy.array([ 
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0],
    ]),
    numpy.array([ 
        [0, 0, 1],
        [0, 0, 1],
        [1, 1, 1],
    ]),
    numpy.array([ 
        [1],
        [1],
        [1],
        [1],
    ]),
    numpy.array([ 
        [1, 1],
        [1, 1],
    ]),
]


def test(x, y, shape):
    h, w = shape.shape

    if x < 0 or x + w > 7 or y + h > cave.shape[0]:
        return False

    return not numpy.any(cave[y:y + h, x:x + w] & shape)

def drop(shape):
    h, w = shape.shape
    x, y = 2, current_y - 3 - h

    while True:
        shift = gas.pop(0)
        gas.append(shift)

        if (test(x + shift, y, shape)):
            x += shift

        if (test(x, y + 1, shape)):
            y += 1
        else:
            break

    cave[y:y + h, x:x + w] |= shape
    return y

# Star 1

In [184]:
for rocks in range(2022):
    current_y = min(current_y, drop(shapes[rocks % 5])) 

cave.shape[0] - current_y

3175

# Start 2, easy way

Just increase cycles, print last row of the map, current rock, wind position and height, find a pattern in text editor and do a simple math

In [185]:
rs = 14173 # Rock at cycle start
re = 15933 # Rock at cycle end

hs = 22083 # Height at cycle start
he = 24820 # Height at cycle end


rl = (1000000000000 - rs) % (re - rs) + rs # Target rock offset
hl = 22415 # Total height at the target rock offset

print('Set hl to the true height at the rock {0}'.format(rl))

((1000000000000 - rs) // (re - rs)) * (he - hs) + hs + (hl - hs)


Set hl to the true height at the rock 14400


1555113636385

# Star 2, cycle detection

Just for curiosity, let's do loop detection

In [186]:
cave = numpy.zeros(shape=(2022*4, 7), dtype=numpy.int8)
gas = list(map(lambda c: ord(c) - 61, data))
current_y = cave.shape[0]

states = {}
current_wind = 0
current_height = 0
cycle_hash = 0

def drop(shape):
    h, w = shape.shape
    x, y = 2, current_y - 3 - h
    wind = 0

    while True:
        shift = gas.pop(0)
        gas.append(shift)
        wind += 1

        if (test(x + shift, y, shape)):
            x += shift

        if (test(x, y + 1, shape)):
            y += 1
        else:
            break

    cave[y:y + h, x:x + w] |= shape
    return y, wind

def get_result(state):
    # Rock at the start and at the end of the cycle 
    rs, re = states[state][0], rocks

    # Height at the start and at the end of the cycle
    hs, he = states[state][1], current_height
    
    # Distance to the target rock
    rl = (1000000000000 - rs) % (re - rs) + rs
    hl = next((states[k][1] for k in states if states[k][0] == rl))

    # Final math
    return ((1000000000000 - rs) // (re - rs)) * (he - hs) + hl
    

for rocks in range(20000):
    if rocks > 0:
        state = hash(tuple(list(cave[current_y, :]) + [rocks % 5, current_wind]))
        position = (rocks, current_height)

        # Cycle looped, get result
        if state == cycle_hash:
            current_height = get_result(state)
            break

        # Cycle found, let it loop again
        if cycle_hash == 0 and state in states:
            cycle_hash = state

        states[state] = position

    y, wind = drop(shapes[rocks % 5])
    current_y = min(y, current_y)

    current_wind = (current_wind + wind) % len(gas)
    current_height = cave.shape[0] - current_y

current_height

1555113636385